# 模型保存

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


## 数据读取

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

## Keras模型保存--保存参数或保存模型结构 + 参数

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model.summary()

# 模型编译，固化模型
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


### 1、保存模型结构 + 参数

In [5]:
# 定义文件夹和文件
logdir = os.path.join('graph_def_and_weights')
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'fashion_mnist_model.h5')

# 定义回调函数
callbacks = [
    keras.callbacks.TensorBoard(log_dir = logdir, profile_batch = 100000000),
    # 定义保存模型
    keras.callbacks.ModelCheckpoint(
        output_model_file, save_best_only = True, save_weights_only = False),
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks=callbacks)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 7s 133us/sample - loss: 0.5436 - accuracy: 0.8077 - val_loss: 0.4384 - val_accuracy: 0.8344
Epoch 2/10
55000/55000 [==============================] - 6s 111us/sample - loss: 0.3931 - accuracy: 0.8585 - val_loss: 0.3657 - val_accuracy: 0.8726
Epoch 3/10
55000/55000 [==============================] - 6s 110us/sample - loss: 0.3552 - accuracy: 0.8710 - val_loss: 0.3435 - val_accuracy: 0.8770
Epoch 4/10
55000/55000 [==============================] - 6s 110us/sample - loss: 0.3299 - accuracy: 0.8804 - val_loss: 0.3451 - val_accuracy: 0.8758
Epoch 5/10
55000/55000 [==============================] - 6s 108us/sample - loss: 0.3109 - accuracy: 0.8867 - val_loss: 0.3318 - val_accuracy: 0.8800
Epoch 6/10
55000/55000 [==============================] - 6s 109us/sample - loss: 0.2933 - accuracy: 0.8933 - val_loss: 0.3184 - val_accuracy: 0.8838
Epoch 7/10
55000/55000 [===========================

#### 载入模型

In [ ]:
loaded_model = keras.models.load_model(output_model_file)
loaded_model.evaluate(x_test_scaled, y_test)

### 2、只保存参数

In [ ]:
# 定义文件夹和文件
logdir = os.path.join('graph_def_and_weights')
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'fashion_mnist_weights.h5')

# 定义回调函数
callbacks = [
    keras.callbacks.TensorBoard(log_dir = logdir, profile_batch = 100000000),
    # 定义保存模型
    keras.callbacks.ModelCheckpoint(
        output_model_file, save_best_only = True, save_weights_only = True),
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks=callbacks)

#### 加载参数
在另一个文件中使用时

In [ ]:
logdir = os.path.join('graph_def_and_weights')
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'fashion_mnist_weights.h5')

model.load_weights(output_model_file)
model.evaluate(x_test_scaled, y_test)

### 3、另一种方法只保存参数

In [ ]:
logdir = os.path.join('graph_def_and_weights')
if not os.path.exists(logdir):
    os.mkdir(logdir)

history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid))

model.save_weights(os.path.join(logdir, 'fashion_mnist_weights_2.h5'))

#### 加载参数
在另一个文件中使用时

In [ ]:
logdir = os.path.join('graph_def_and_weights')
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'fashion_mnist_weights_2.h5')

model.load_weights(output_model_file)
model.evaluate(x_test_scaled, y_test)

## Keras模型转化为SavedModel

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model.summary()

# 模型编译，固化模型
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 7s 126us/sample - loss: 0.5293 - accuracy: 0.8130 - val_loss: 0.4131 - val_accuracy: 0.8542
Epoch 2/10
55000/55000 [==============================] - 6s 108us/sample - loss: 0.3923 - accuracy: 0.8595 - val_loss: 0.3637 - val_accuracy: 0.8720
Epoch 3/10
55000/55000 [==============================] - 6s 112us/sample - loss: 0.3530 - accuracy: 0.8738 - val_loss: 0.3747 - val_accuracy: 0.8614
Epoch 4/10
55000/55000 [==============================] - 6s 110us/sample - loss: 0.3291 - accuracy: 0.8823 - val_loss: 0.3323 - val_accuracy: 0.8774
Epoch 5/10
55000/55000 [==============================] - 6s 118us/sample - loss: 0.3096 - accuracy: 0.8880 - val_loss: 0.3218 - val_accuracy: 0.8878
Epoch 6/10
55000/55000 [==============================] - 6s 115us/sample - loss: 0.2934 - accuracy: 0.8938 - val_loss: 0.3150 - val_accuracy: 0.8856
Epoch 7/10
55000/55000 [===========================

### 保存模型

In [8]:
logdir = os.path.join('keras_saved_graph')
if not os.path.exists(logdir):
    os.mkdir(logdir)

tf.saved_model.save(model, logdir)

INFO:tensorflow:Assets written to: keras_saved_graph\assets


### 使用工具查看和测试

In [9]:
!saved_model_cli show --dir ./keras_saved_graph --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_1_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28)
        name: serving_default_flatten_1_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


2020-02-10 12:35:17.031604: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll


In [20]:
!saved_model_cli show --dir ./keras_saved_graph --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_1_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_1_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_2'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


2020-02-10 12:43:58.013512: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll


In [2]:
!saved_model_cli run --dir ./keras_saved_graph --tag_set serve --signature_def serving_default \
    --input_exprs "flatten_1_input=np.ones((2, 28, 28))"

Result for output key dense_2:
[[0.07866267 0.04084459 0.0750244  0.03414746 0.09606657 0.01821422
  0.09031539 0.03132759 0.52319443 0.0122027 ]
 [0.07866267 0.04084459 0.0750244  0.03414746 0.09606657 0.01821422
  0.09031539 0.03132759 0.52319443 0.0122027 ]]


2020-02-10 12:52:31.779954: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll
2020-02-10 12:52:34.206038: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2020-02-10 12:52:34.773939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 850M major: 5 minor: 0 memoryClockRate(GHz): 0.9015
pciBusID: 0000:0a:00.0
2020-02-10 12:52:34.774145: I tensorflow/stream_executor/platform/default/dlopen_checker_stub.cc:25] GPU libraries are statically linked, skip dlopen check.
2020-02-10 12:52:34.775123: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1746] Adding visible gpu devices: 0
2020-02-10 12:52:34.775552: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2020-02-10 12:52:34.778724: I tensorflow/core/common_runtime/gp

### 使用程序进行验证和推理

In [4]:
# 加载模型
loaded_saved_model = tf.saved_model.load('./keras_saved_graph')
# 列出所有签名函数
print(list(loaded_saved_model.signatures.keys()))

['serving_default']


In [5]:
# 利用签名获得函数句柄
inference = loaded_saved_model.signatures['serving_default']
print(inference)

In [6]:
# 打印输出结构
print(inference.structured_outputs)

{'dense_2': TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_2')}


In [7]:
# 使用inference进行推理
results = inference(tf.constant(x_test_scaled[0:1]))
print(results)
print(results['dense_2'])
print(results['dense_2'].numpy())

{'dense_2': <tf.Tensor: id=622, shape=(1, 10), dtype=float32, numpy=
array([[1.8168412e-06, 8.1950509e-07, 9.4149618e-06, 7.7796500e-07,
        1.4993745e-05, 1.9446755e-02, 2.9783282e-06, 3.8230207e-02,
        2.2591450e-04, 9.4206625e-01]], dtype=float32)>}
tf.Tensor(
[[1.8168412e-06 8.1950509e-07 9.4149618e-06 7.7796500e-07 1.4993745e-05
  1.9446755e-02 2.9783282e-06 3.8230207e-02 2.2591450e-04 9.4206625e-01]], shape=(1, 10), dtype=float32)
[[1.8168412e-06 8.1950509e-07 9.4149618e-06 7.7796500e-07 1.4993745e-05
  1.9446755e-02 2.9783282e-06 3.8230207e-02 2.2591450e-04 9.4206625e-01]]
